In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
from pathlib import Path
from shapely.geometry import Point, Polygon

In [ ]:
datapath = Path('data/geopandas/')

# GeoPandas - dataframes with geometry for GIS applications

## [Nice overview tutorial](https://geopandas.org/en/stable/getting_started/introduction.html)
## [Examples Gallery](https://geopandas.org/en/stable/gallery/index.html)

# OVERVIEW
***

### get some data - `read_file` is the ticket for GeoJSON, shapefiles, GDB, etc.

In [ ]:
parks = gp.read_file(datapath / 'Madison_Parks.geojson')

## writing back out is veeeery similar with `to_file` but give a few options for formats

In [ ]:
parks.to_file(datapath / "parks.shp")

In [ ]:
# or with a different driver
parks.to_file(datapath / "parks.json", driver='GeoJSON')

## this now looks like a Pandas DataFrame but there's a special column `geometry`

In [ ]:
parks.head()

### also some important metadata particularly the [CRS](https://en.wikipedia.org/wiki/Spatial_reference_system)

In [ ]:
parks.crs

> ## pro tip: You can have multiple geometry columns but only one is _active_ -- this is important later as we do operations on GeoDataFrames. The column labeled `geometry` is typically the active one but you [you can change it](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html).

## So what's up with these geometries? They are represented as [`shapely`](https://shapely.readthedocs.io/en/stable/manual.html) objects so can be:
- ### polygon / multi-polygon
- ### point / multi-point
- ### line / multi-line

## we can access with pandas `loc` and `iloc` references

In [ ]:
parks.iloc[1].geometry.type

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.type

### There are other cool shapely properties like `area`

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.area

### ruh-roh - what's up with this CRS and tiny area number?

In [ ]:
parks.crs

## area units in lat/long don't make sense. Let's project to something in meters (but how?) `to_crs` will do it, but importantly, either reassign or set `inplace=True`

In [ ]:
parks.to_crs(3071, inplace=True)
parks.crs

In [ ]:
parks.loc[parks.ShortName=='Olin'].geometry.area

### there are loads of useful methods for `shapely` objects for relationships between geometries (intersection, distance, etc.) but we will skip these for now because GeoPandas facilitates these things for entire geodataframes! #sick

# VISUALIZATION
***

## So back to GeoDataFrames.....we can look at them spatially as well with `plot()`

In [ ]:
parks.plot()

## easily make a chloropleth map using a selected column as the color (and add a legend) using `plot()`

In [ ]:
parks.columns

In [ ]:
parks.sample(4)

In [ ]:
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6})

## also a very cool interactive plot options with a basemap using `explore()`

In [ ]:
parks.explore(column='Acreage', cmap='magma')

## we can read in another shapefile

In [ ]:
hoods = gp.read_file(datapath / 'Neighborhood_Associations.geojson')

In [ ]:
hoods.sample(5)

### and we can plot these on top of each other

In [ ]:
ax_hood = hoods.plot()
# now plot the other one but specify which axis to plot on (ax=ax_hood)
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, ax=ax_hood)

## WAT! Why so far apart?

In [ ]:
hoods.crs

## we need to reproject. Geopandas uses `to_crs()` for this purpose

In [ ]:
# we can reproject, and set hoods to park crs 
hoods.to_crs(parks.crs, inplace=True)

In [ ]:
ax_hood = hoods.plot()
# now plot the other one but specify which axis to plot on (ax=ax_hood)
parks.plot(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, ax=ax_hood)

## or similarly with the interactive maps

In [ ]:
m_hood = hoods.explore()
parks.explore(column="Acreage", cmap = 'magma', k=5, legend=True, legend_kwds={'shrink': 0.6}, m=m_hood)

## we can make a new geodataframe using shapely properties of the geometry - how about centroids?

## TEST YOUR SKILLS #0
- make a new geodataframe of the parks
- add a columns with centroids for each park
- plot an interactive window with the park centroids and the neighborhoods
- hints: 
    - remember the shapely methods are available for each geometry object (e.g. `centroid()`) 
    - you can loop over the column in a couple different ways
    - you can define which columns contains the geometry of a geodataframe
    - you will likely have to define the CRS

# GEOSPATIAL OPERATIONS
***

# Operations on and among geodataframes...do I need to use a GIS program?

## Dissolve

In [ ]:
hoods.dissolve() # note it defaults to filling all the columns with the first value

In [ ]:
ax=hoods.dissolve().plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

## Convex Hull

In [ ]:
ax = hoods.dissolve().convex_hull.plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

## Bounding Box is a little more tricky

In [ ]:
hoods.bounds # that's per each row

In [ ]:
tb = hoods.total_bounds # this gives overall bounds
tb

## We can make a polygon from these coordinates with `shapely`

In [ ]:
from shapely.geometry import box

In [ ]:
bbox = box(tb[0], tb[1], tb[2], tb[3])
# pro tip - when passing a bunch of ordered arguments, '*' will unpack them #nice
bbox = box(*hoods.total_bounds)

## to make a GeoDataFrame from scratch, the minimum you need is geometry, but a crs is important, and some data will populate more columns

In [ ]:

hoods_boundary = gp.GeoDataFrame(data={'thing':['bounding_box']},geometry=[bbox], crs=hoods.crs)
hoods_boundary

In [ ]:
ax = hoods_boundary.plot()
hoods.plot(facecolor=None, edgecolor='orange', ax=ax)

# How about some spatial joins?

## we can bring in information based on locational overlap. Let's look at just a couple neighborhoods (Marquette and Tenny-Lapham) on the Isthmus

In [ ]:
isthmus = hoods.loc[hoods['NEIGHB_NAME'].str.contains('Marquette') | 
                   hoods['NEIGHB_NAME'].str.contains('Tenney')]
isthmus

In [ ]:
isthmus.explore()

In [ ]:
isthmus.sjoin(parks).explore()

In [ ]:
parks.sjoin(isthmus).explore()

### so, it matters which direction you join from. The geometry is preserved from the dataframe "on the left"
### equivalently, you can be more explicit in calling `sjoin`

In [ ]:
gp.sjoin(left_df=parks, right_df=isthmus).explore()

In [ ]:
isthmus_parks = gp.sjoin(left_df=parks, right_df=isthmus)

## we are going to use this `isthmus_parks` geoDataFrame a little later, but we want to trim out some unneeded and distracting columns. We can use `.drop()` just like with a regular Pandas DataFrame

In [ ]:
isthmus_parks.columns

In [ ]:
isthmus_parks.drop(columns=[ 'index_right','OBJECTID_right', 'NA_ID', 'STATUS', 'CLASSIFICA', 'Web',
       'ShapeSTArea', 'ShapeSTLength'], inplace=True)

# Let's explore the various predicates with a small intersecting box

In [ ]:
bbox = box(570600, 290000, 573100, 291700)
bounds = gp.GeoDataFrame(geometry=[bbox],crs=parks.crs)
bounds.plot()

## See [documentation](https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates) for full set of options for predicates. We'll just check out a couple options: `intersects`, `contains`, `within`

# TEST YOUR SKILLS #1
Using the `bounds` geodataframe you just made, write a function to visualize predicate behaviors.
- your function should accept a left geodataframe, a right geodataframe, and a string for the predicate
- your function should plot:
    - the left geodataframe in (default) blue
    - the result of the spatial join operation in another color
    - the right geodataframe in another color with outline only
- then you should set the title of the plot to the string predicate value used
- the geodataframes to test with are `isthmus_parks` and `bounds`
- your function should `return` the joined geodataframe

- a couple hints:
    - in the `plot` method are a couple args called `facecolor` and `edgecolor` that will help plot the rectangle
    - there are other predicates to try out 

- _advanced options_: if that was easy, you can try a couple other things like:
    - explore joins with points and lines in addition to just polygons
    - change around the `bounds` polygon dimensions 
    - use `explore()` to make an interactive map

# Spatial joins are particularly useful with collections of points. A common case is to add a polygon attribute to points falling within each polygon. Let's check out a bigger point dataset with all the trees on streets in Madison

In [ ]:
trees = gp.read_file(datapath / 'Street_Trees.geojson', index_col=0)
trees.plot(column='SPECIES')

## let's put this into the same crs as neighborhoods and join the data together so we can have a neighborhood attribute on the trees geodataframe

In [ ]:
trees.to_crs(hoods.crs, inplace=True)

In [ ]:
hoods.columns

## NOTE: if we pass only some columns of the GeoDataFrame, only those columns will be included in the result, which is cool. _But_ - must include the active geometry column as well!

In [ ]:
trees_with_hoods = trees[['SPECIES','DIAMETER','geometry']].sjoin(hoods[['NEIGHB_NAME','geometry']])
trees_with_hoods

# now we can do a groupby, for example, to find things like the average or max diameter of trees in each neighborhood

In [ ]:
trees_with_hoods.groupby('NEIGHB_NAME')['DIAMETER'].max().plot(kind='bar')

# We could rearrange that bar chart in various ways, but we can also flip this back to the original neighborhoods GeoDataFrame to make a more useful spatial plot. Note that we used the spatial join to join together the attribute "Neighborhood Name" with each tree point. But now, we can aggregate those results and assign them based on an attribute rather than geospatially just like regular Pandas DataFrames

In [ ]:
hood_trees = hoods.copy()
tree_summary = trees_with_hoods.groupby('NEIGHB_NAME')['DIAMETER'].max()
hood_trees.merge(tree_summary,
                left_on = 'NEIGHB_NAME', right_on='NEIGHB_NAME').explore(column="DIAMETER")

## As we've seen, spatial joins are powerful, but they really only gather data from multiple collections. What if we want to actually calculate the amount of overlap among shapes? Or create new shapes based on instersection or not intersection of shapes? [`overlay`](https://geopandas.org/en/stable/docs/user_guide/set_operations.html?highlight=overlay) does these things.

## main options are `intersection', '

In [ ]:
inters = bounds.overlay(isthmus, how='intersection')
inters.plot()

In [ ]:
inters

In [ ]:
differ = bounds.overlay(isthmus, how='difference')
differ.plot()

In [ ]:
differ

In [ ]:
unite = bounds.overlay(isthmus, how='union')
unite.plot()

In [ ]:
unite

In [ ]:
symdiff = bounds.overlay(isthmus, how='symmetric_difference')
symdiff.plot()

In [ ]:
symdiff

# On your own...
- what if you switch left and right dataframes?
- how can you evaluate the areas of overlap for the intersection case?